In [1]:
import mlflow

# Asegurar que estamos conectados a MLflow
mlflow.set_tracking_uri("http://10.43.101.205:5000")
mlflow.set_experiment("penguins_experiment")


# Activar Auto Logging para Sklearn
mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True)

In [4]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "admin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "supersecret"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://10.43.101.205:9000"  # Ajusta la IP si es necesario

import boto3
s3 = boto3.client(
    "s3",
    endpoint_url=os.environ["MLFLOW_S3_ENDPOINT_URL"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"]
)

bucket_name = "mlflows3"

In [5]:
import pandas as pd
from io import StringIO

# Descargar el archivo desde MinIO
obj = s3.get_object(Bucket=bucket_name, Key="penguins_size.csv")
data = obj["Body"].read().decode("utf-8")  # Decodificar a texto

# Convertirlo a un DataFrame de pandas
df = pd.read_csv(StringIO(data))

import mlflow

# Iniciar un run en MLflow
mlflow.set_tracking_uri("http://10.43.101.205:5000")  # Ajusta la IP si es diferente
mlflow.set_experiment("penguins_experiment")

with mlflow.start_run(run_name="load_penguins_data"):
    # Guardar el dataset en MinIO como un artefacto de MLflow
    df.to_csv("penguins_size_processed.csv", index=False)
    mlflow.log_artifact("penguins_size_processed.csv")
    print("✅ Dataset registrado en MLflow correctamente.")
    

✅ Dataset registrado en MLflow correctamente.


In [6]:
# Mostrar las primeras filas para confirmar que se cargó bien
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [7]:
import boto3
import pandas as pd
from io import StringIO
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# ✅ Etapa 1: Procesamiento de Datos

# Eliminar valores nulos con la estrategia de imputación más frecuente
imputer = SimpleImputer(strategy='most_frequent')
df.iloc[:, :] = imputer.fit_transform(df)

# Verificar que no haya valores nulos
print("Valores nulos después de imputación:", df.isna().sum())

# Convertir variables categóricas a numéricas usando Label Encoding
label_encoders = {}
for col in ['island', 'sex', 'species']:  # Variables categóricas
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Guardamos los encoders por si se necesitan después

# ✅ Definir experimento en MLflow
mlflow.set_tracking_uri("http://10.43.101.205:5000")  # Ajusta la IP si es diferente
mlflow.set_experiment("penguins_experiment")

# Separar características y variable objetivo
X = df.drop(columns=["species"])  # "species" es la variable objetivo
y = df["species"]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Iniciar un run en MLflow
with mlflow.start_run(run_name="DecisionTree_penguins"):

    # Entrenar modelo
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Evaluar el modelo
    acc = accuracy_score(y_test, y_pred)
    print(f"✅ Accuracy: {acc}")
    print(classification_report(y_test, y_pred))

    # Registrar métricas en MLflow
    mlflow.log_metric("accuracy", acc)

    # Registrar el modelo en MLflow
    mlflow.sklearn.log_model(model, "penguins_model")

    print("✅ Modelo registrado en MLflow.")



Valores nulos después de imputación: species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64


2025/03/17 19:00:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.9/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


✅ Accuracy: 0.9565217391304348
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       0.89      1.00      0.94        16
           2       0.95      0.95      0.95        21

    accuracy                           0.96        69
   macro avg       0.95      0.96      0.95        69
weighted avg       0.96      0.96      0.96        69

✅ Modelo registrado en MLflow.


In [8]:
import boto3
import pandas as pd
from io import StringIO
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

### ✅ Entrenamiento con XGBoost
with mlflow.start_run(run_name="XGBoost_penguins"):

    # Entrenar modelo
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42)
    xgb_model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred_xgb = xgb_model.predict(X_test)

    # Evaluar el modelo
    acc_xgb = accuracy_score(y_test, y_pred_xgb)
    print(f"✅ XGBoost Accuracy: {acc_xgb}")
    print(classification_report(y_test, y_pred_xgb))

    # Registrar métricas en MLflow
    mlflow.log_metric("accuracy_xgb", acc_xgb)

    # Registrar el modelo en MLflow
    mlflow.sklearn.log_model(xgb_model, "penguins_xgboost")

    print("✅ XGBoost registrado en MLflow.")

/usr/local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:00:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ XGBoost Accuracy: 0.9710144927536232
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       0.89      1.00      0.94        16
           2       1.00      1.00      1.00        21

    accuracy                           0.97        69
   macro avg       0.96      0.98      0.97        69
weighted avg       0.97      0.97      0.97        69

✅ XGBoost registrado en MLflow.


# Ejecuciones variando hiperparámetros

In [10]:
# ✅ Lista de hiperparámetros para 20 ejecuciones
param_grid = [
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01},
    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.01},
    {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.01},
    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.01},
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.05},
    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.05},
    {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05},
    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05},
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.1},
    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1},
    {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.1},
    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1},
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.2},
    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.2},
    {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.2},
    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.2},
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.3},
    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.3},
    {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.3},
    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.3},
]

# ✅ Ejecutar 20 experimentos con MLflow (modificado)
for i, params in enumerate(param_grid, 1):
    with mlflow.start_run(run_name=f"XGBoost_run_{i}"):
        print(f"🔄 Ejecutando experimento {i} con {params}")

        # Entrenar modelo con los hiperparámetros
        xgb_model = XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            use_label_encoder=False,
            eval_metric="mlogloss",
            random_state=42
        )
        xgb_model.fit(X_train, y_train)

        # Hacer predicciones
        y_pred = xgb_model.predict(X_test)

        # Evaluar el modelo
        acc = accuracy_score(y_test, y_pred)
        print(f"✅ Accuracy: {acc}")
        print(classification_report(y_test, y_pred))

        # Registrar hiperparámetros en MLflow
        mlflow.log_param("n_estimators", params['n_estimators'])
        mlflow.log_param("max_depth", params['max_depth'])
        mlflow.log_param("learning_rate", params['learning_rate'])

        # Registrar métricas en MLflow
        mlflow.log_metric("accuracy", acc)

        # Registrar el modelo en MLflow con un nombre de artefacto constante
        mlflow.sklearn.log_model(xgb_model, "penguins_xgboost_model")

        print(f"✅ Experimento {i} registrado en MLflow.")


🔄 Ejecutando experimento 1 con {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
✅ Accuracy: 0.9710144927536232
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.94      0.94      0.94        16
           2       1.00      1.00      1.00        21

    accuracy                           0.97        69
   macro avg       0.97      0.97      0.97        69
weighted avg       0.97      0.97      0.97        69



/usr/local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:10:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Experimento 1 registrado en MLflow.
🔄 Ejecutando experimento 2 con {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.01}


/usr/local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:10:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Accuracy: 0.9710144927536232
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.94      0.94      0.94        16
           2       1.00      1.00      1.00        21

    accuracy                           0.97        69
   macro avg       0.97      0.97      0.97        69
weighted avg       0.97      0.97      0.97        69

✅ Experimento 2 registrado en MLflow.
🔄 Ejecutando experimento 3 con {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.01}


...

✅ Experimento 18 registrado en MLflow.
🔄 Ejecutando experimento 19 con {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.3}


/usr/local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:11:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Accuracy: 0.9710144927536232
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       0.89      1.00      0.94        16
           2       1.00      1.00      1.00        21

    accuracy                           0.97        69
   macro avg       0.96      0.98      0.97        69
weighted avg       0.97      0.97      0.97        69

✅ Experimento 19 registrado en MLflow.
🔄 Ejecutando experimento 20 con {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.3}
✅ Accuracy: 0.9710144927536232
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       0.89      1.00      0.94        16
           2       1.00      1.00      1.00        21

    accuracy                           0.97        69
   macro avg       0.96      0.98      0.97        69
weighted avg       0.97      0.97      0.97        69



/usr/local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:11:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Experimento 20 registrado en MLflow.


# Selección del mejor modelo

In [11]:
import mlflow
import pandas as pd

# Definir la URI de MLflow (asegúrate de que coincide con la configuración)
mlflow.set_tracking_uri("http://10.43.101.205:5000")

# Obtener todos los runs del experimento "penguins_experiment"
experiment = mlflow.get_experiment_by_name("penguins_experiment")
df_runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Filtrar las columnas relevantes
df_results = df_runs[["run_id", "params.n_estimators", "params.max_depth", "params.learning_rate", "metrics.accuracy"]]

# Ordenar por accuracy descendente
df_results = df_results.sort_values(by="metrics.accuracy", ascending=False)

# Mostrar el mejor resultado
print("🏆 Mejor modelo:")
print(df_results.head(1))  # Muestra solo el mejor

# Seleccionar el run con mejor accuracy
best_run_id = df_results.head(1)["run_id"].iloc[0]
print("🏆 Mejor run_id:", best_run_id)

# Construir la URI del modelo. 'penguins_xgboost_model' es el nombre constante usado en log_model.
model_uri = f"runs:/{best_run_id}/penguins_xgboost_model"

# Registrar el modelo en el Model Registry con el nombre "XGBoost_Best_Deployed"
registered_model = mlflow.register_model(model_uri, "XGBoost_Best_Deployed")
print("✅ Modelo registrado correctamente:", registered_model)

Registered model 'XGBoost_Best_Deployed' already exists. Creating a new version of this model...
2025/03/17 19:11:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoost_Best_Deployed, version 3


🏆 Mejor modelo:
                             run_id params.n_estimators params.max_depth  \
2  e1c7ea2c7bf74750807b05d8f061417e                 100                4   

  params.learning_rate  metrics.accuracy  
2                  0.3          0.985507  
🏆 Mejor run_id: e1c7ea2c7bf74750807b05d8f061417e
✅ Modelo registrado correctamente: <ModelVersion: aliases=[], creation_timestamp=1742238711268, current_stage='None', description='', last_updated_timestamp=1742238711268, name='XGBoost_Best_Deployed', run_id='e1c7ea2c7bf74750807b05d8f061417e', run_link='', source='s3://mlflows3/artifacts/1/e1c7ea2c7bf74750807b05d8f061417e/artifacts/penguins_xgboost_model', status='READY', status_message='', tags={}, user_id='', version='3'>


Created version '3' of model 'XGBoost_Best_Deployed'.


In [12]:
model = mlflow.pyfunc.load_model("models:/XGBoost_Best_Deployed/latest")
print("✅ Modelo cargado correctamente desde MLflow.")

✅ Modelo cargado correctamente desde MLflow.


In [13]:
print(label_encoders["species"].classes_)

['Adelie' 'Chinstrap' 'Gentoo']


In [14]:
# Imprimir el mapeo de las columnas categóricas
print("Mapping para 'island':", dict(zip(label_encoders['island'].classes_, range(len(label_encoders['island'].classes_)))))
print("Mapping para 'sex':", dict(zip(label_encoders['sex'].classes_, range(len(label_encoders['sex'].classes_)))))
print("Mapping para 'species':", dict(zip(label_encoders['species'].classes_, range(len(label_encoders['species'].classes_)))))


Mapping para 'island': {'Biscoe': 0, 'Dream': 1, 'Torgersen': 2}
Mapping para 'sex': {'.': 0, 'FEMALE': 1, 'MALE': 2}
Mapping para 'species': {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
